## Configuartion

In [1]:
%load_ext autoreload

import os
import sys

home = os.getenv("HOME")
use_gdrive = False

## Setup Directories

In [2]:
ds_to_use = ['HP1-FvM']

local_nlp_base_dir = f'{home}/shared'
local_data_base_dir = f'{local_nlp_base_dir}/NLP-Data'
local_data_dir = f'{local_data_base_dir}/audio'
extern_nlp_base_dir = None
extern_data_base_dir = None
extern_data_dir = None

runs_on_colab = (home == '/root')
print( f'runs on colab: {runs_on_colab}')

if not runs_on_colab:
    os.environ['http_proxy'] = 'http://192.168.8.50:3128'
    os.environ['https_proxy'] = 'http://192.168.8.50:3128'
else:
    # to get access to the datasets we use gdrive
    use_gdrive = True
    # install packages
    !pip install jiwer
    # create local directories
    !mkdir $local_nlp_base_dir
    !mkdir $local_data_base_dir

if use_gdrive:
    gdrive_base = '/content/gdrive'    
    extern_nlp_base_dir = f'{gdrive_base}/MyDrive'
    extern_data_base_dir = f'{extern_nlp_base_dir}/NLP-Data'
    extern_data_dir = f'{extern_data_base_dir}/audio'

    if not os.path.isdir(gdrive_base):
        from google.colab import drive
        drive.mount(gdrive_base)

if not os.path.isdir(local_data_dir):
    !mkdir $local_data_dir

if extern_nlp_base_dir:
    model_dir = f'{extern_nlp_base_dir}/NLP-Models/GermanWave2Vec'
else:
    model_dir = f'{local_nlp_base_dir}/NLP-Models/GermanWave2Vec'

local_gitviews=f'{local_nlp_base_dir}/gitviews'
!mkdir $local_gitviews

git_views_dir = f'{local_nlp_base_dir}/gitviews/extern'
git_view_path = f'{git_views_dir}/GermanWave2Vec'

if not os.path.isdir(git_views_dir):
    !mkdir $git_views_dir
    !cd $git_views_dir; git clone https://github.com/ElUnrast/GermanWave2Vec.git

runs on colab: False
mkdir: cannot create directory ‘/home/ki-mo/shared/gitviews’: File exists


In [3]:
if runs_on_colab:
    git_view_path = f'{git_views_dir}/GermanWave2Vec'
    # !cd $git_view_path; git fetch --all; git reset --hard origin/main
    !cd $git_view_path; git fetch origin main; git reset --hard origin/main

script_path = f'{git_views_dir}/GermanWave2Vec/python'
sys.path.insert(0, script_path)

## Install packages and do Imports

In [4]:
import numpy as np
import pandas as pd
import sklearn
import jiwer
from jiwer import wer
from tqdm.notebook import tqdm_notebook
from sklearn.model_selection import train_test_split

In [5]:
#  %aimport SnippetDatasets
from SnippetDatasets import SnippetDatasets, calc_wer
from GermanSpeechDatasetWidgetFactory import GermanSpeechDatasetWidgetFactory

## Initialize Helper Classes

In [6]:
my_datasets = SnippetDatasets(
    runs_on_colab, 
    local_audio_base_dir=local_data_dir, 
    git_repository=git_view_path,
    extern_audio_base_dir=extern_data_dir
)

## Choose Dataset

In [7]:
widget_factory = GermanSpeechDatasetWidgetFactory(my_datasets, ds_to_use)
print('Please choose Datasets to use')
ds_checkboxes_widget = widget_factory.create_dataset_choice_widget()
display(ds_checkboxes_widget)

Please choose Datasets to use


Box(children=(Checkbox(value=True, description='HP1-FvM', indent=False), Checkbox(value=False, description='HP…

In [8]:
display(widget_factory.create_validation_tab_widget())


Validate
-----------------------------
Loading Dataset: HP1-FvM - content-translated-with_original.csv
Pruning Dataset HP1-FvM with 8055 Entries
 - 8055 Entries left after Length Cut (min=31, max=4000)
 - 8055 Entries left after Action Cut
Use Snipped Directory: /home/ki-mo/shared/NLP-Data/audio/NLP - Harry Potter/HP1-FvM - 8055/3868 Wrong


In [9]:
print(script_path)

/home/ki-mo/shared/gitviews/extern/GermanWave2Vec/python
